## 1. This Jupyter Notebook shows the initial process, using the overall rank as target and all other usable fields as features.

In [1]:
import pandas as pd
import numpy as np
from path import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import r2_score


In [2]:
# Loading data
file_path = Path("Resources/2022-College-Guide-Main-Rankings-clear.csv")
df_rankings = pd.read_csv(file_path)
df_rankings.head()


,UnitID,Rank,Name,8-year graduation rate,Graduation rate rank,"Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.",Graduation rate performance rank,Pell/non-Pell graduation gap,Pell graduation gap rank,Number of Pell graduates,...,% of federal work-study funds spent on service,% of federal work-study funds spent on service rank,Earns Carnegie Community Engagement Classification?,Voting engagement points,% of grads with service-oriented majors,Service-oriented majors rank,Social mobility rank,Research rank,Service rank,Unnamed: 40
0,243744,1,Stanford University (CA),0.963302,6,0.949654,187,-0.021703,46,260.6667,...,0.290903,10,NO,3,0.068361,401,3,2,187,NaN
1,215062,2,University of Pennsylvania (PA),0.964294,5,0.938620,149,-0.041917,104,376.6667,...,0.160573,71,YES,6,0.246743,209,2,9,40,NaN
2,166683,3,MA Institute of Technology (MA),0.950971,10,0.990213,334,-0.032985,72,182.6667,...,0.065477,332,NO,2,0.000000,436,5,1,324,NaN
3,186131,4,Princeton University (NJ),0.976861,2,0.947812,137,-0.052740,132,153.3333,...,0.064500,339,NO,6,0.000000,436,7,6,224,NaN
4,198419,5,Duke University (NC),0.954474,9,0.967952,267,-0.014278,36,231.6667,...,0.089523,224,YES,5,0.170164,305,12,15,20,NaN


In [3]:
df_rankings.columns

Index(['UnitID', 'Rank', 'Name', '8-year graduation rate',
       'Graduation rate rank',
       'Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.',
       'Graduation rate performance rank', 'Pell/non-Pell graduation gap',
       'Pell graduation gap rank', 'Number of Pell graduates',
       'Actual vs. predicted Pell enrollment', 'Pell performance rank',
       'Median earnings 10 years after entering college',
       'Predicted median earnings 10 years after entering college',
       'Earnings performance rank',
       'Net price of attendance for families below $75,000 income',
       'Net price rank', '% of loan principal remaining 5 years later',
       'Repayment rank', 'Predicted principal remaining',
       'Repayment rate performance rank', 'Research expenditures, in millions',
       'Research expenditures rank', 'Bachelor's to PhD rank',
       'Science & engineering PhDs awarded', 'Science & engineering PhDs rank',
       'Faculty receiving sign

In [4]:
df_rankings.dtypes

UnitID                                                                            int64
Rank                                                                              int64
Name                                                                             object
8-year graduation rate                                                          float64
Graduation rate rank                                                              int64
Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.    float64
Graduation rate performance rank                                                  int64
Pell/non-Pell graduation gap                                                    float64
Pell graduation gap rank                                                          int64
Number of Pell graduates                                                        float64
Actual vs. predicted Pell enrollment                                            float64
Pell performance rank           

In [5]:
df_rankings.count()

UnitID                                                                          442
Rank                                                                            442
Name                                                                            442
8-year graduation rate                                                          442
Graduation rate rank                                                            442
Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.    442
Graduation rate performance rank                                                442
Pell/non-Pell graduation gap                                                    442
Pell graduation gap rank                                                        442
Number of Pell graduates                                                        442
Actual vs. predicted Pell enrollment                                            442
Pell performance rank                                                       

In [6]:
# Drop last column
df_rankings.drop('Unnamed: 40', axis=1, inplace=True)

In [7]:
# Encoding Carnegie column (0=NO, 1=YES)

le = LabelEncoder()
df2_rankings = df_rankings.copy()
df2_rankings['Earns Carnegie Community Engagement Classification?'] = le.fit_transform(df2_rankings['Earns Carnegie Community Engagement Classification?'])


In [8]:
df2_rankings.head()

,UnitID,Rank,Name,8-year graduation rate,Graduation rate rank,"Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.",Graduation rate performance rank,Pell/non-Pell graduation gap,Pell graduation gap rank,Number of Pell graduates,...,ROTC rank,% of federal work-study funds spent on service,% of federal work-study funds spent on service rank,Earns Carnegie Community Engagement Classification?,Voting engagement points,% of grads with service-oriented majors,Service-oriented majors rank,Social mobility rank,Research rank,Service rank
0,243744,1,Stanford University (CA),0.963302,6,0.949654,187,-0.021703,46,260.6667,...,308,0.290903,10,0,3,0.068361,401,3,2,187
1,215062,2,University of Pennsylvania (PA),0.964294,5,0.938620,149,-0.041917,104,376.6667,...,250,0.160573,71,1,6,0.246743,209,2,9,40
2,166683,3,MA Institute of Technology (MA),0.950971,10,0.990213,334,-0.032985,72,182.6667,...,12,0.065477,332,0,2,0.000000,436,5,1,324
3,186131,4,Princeton University (NJ),0.976861,2,0.947812,137,-0.052740,132,153.3333,...,34,0.064500,339,0,6,0.000000,436,7,6,224
4,198419,5,Duke University (NC),0.954474,9,0.967952,267,-0.014278,36,231.6667,...,45,0.089523,224,1,5,0.170164,305,12,15,20


In [9]:
# Define the features set.
X = df2_rankings.copy()

# Dropping target columns and unique identifier columns, etc
X = X.drop(columns=["UnitID", "Rank", "Name"])
X.head()


,8-year graduation rate,Graduation rate rank,"Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.",Graduation rate performance rank,Pell/non-Pell graduation gap,Pell graduation gap rank,Number of Pell graduates,Actual vs. predicted Pell enrollment,Pell performance rank,Median earnings 10 years after entering college,...,ROTC rank,% of federal work-study funds spent on service,% of federal work-study funds spent on service rank,Earns Carnegie Community Engagement Classification?,Voting engagement points,% of grads with service-oriented majors,Service-oriented majors rank,Social mobility rank,Research rank,Service rank
0,0.963302,6,0.949654,187,-0.021703,46,260.6667,0.038096,122,97798,...,308,0.290903,10,0,3,0.068361,401,3,2,187
1,0.964294,5,0.938620,149,-0.041917,104,376.6667,-0.000590,211,103246,...,250,0.160573,71,1,6,0.246743,209,2,9,40
2,0.950971,10,0.990213,334,-0.032985,72,182.6667,0.076769,65,111222,...,12,0.065477,332,0,2,0.000000,436,5,1,324
3,0.976861,2,0.947812,137,-0.052740,132,153.3333,0.062997,85,95689,...,34,0.064500,339,0,6,0.000000,436,7,6,224
4,0.954474,9,0.967952,267,-0.014278,36,231.6667,0.028513,142,93115,...,45,0.089523,224,1,5,0.170164,305,12,15,20


In [10]:
# Checking datatypes to make sure there are no strings
X.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 37 columns):
 #   Column                                                                        Non-Null Count  Dtype  
---  ------                                                                        --------------  -----  
 0   8-year graduation rate                                                        442 non-null    float64
 1   Graduation rate rank                                                          442 non-null    int64  
 2   Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.  442 non-null    float64
 3   Graduation rate performance rank                                              442 non-null    int64  
 4   Pell/non-Pell graduation gap                                                  442 non-null    float64
 5   Pell graduation gap rank                                                      442 non-null    int64  
 6   Number of Pell graduates          

In [11]:
# Define the target set.
y = df2_rankings["Rank"].ravel()
y[:5]


array([1, 2, 3, 4, 5], dtype=int64)

In [12]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Creating a StandardScaler instance.
scaler = StandardScaler()

# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [13]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (331, 37)
Training Labels Shape: (331,)
Testing Features Shape: (111, 37)
Testing Labels Shape: (111,)


## Random Forest Regressor

In [14]:
# Instantiate model with 1000 decision trees
rf_model = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf_model=rf_model.fit(X_train_scaled, y_train)

In [15]:
# Making predictions using the testing data.
rf_predictions = rf_model.predict(X_test_scaled)


In [16]:
# Evaluate the performance with r2 score
r2_rf= r2_score(y_test,rf_predictions)
print('The r2 score for this Random Forest Regressor model is:', r2_rf)
#r2 score with 1000 estimators/ random state 42 = .964; 500 estimators .963; 1500 estimators .964

The r2 score for this Random Forest Regressor model is: 0.964116868404521


In [17]:
# Getting accuracy data with Regressor/continuous variable model

# Calculate the absolute errors
errors = abs(rf_predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')


Mean Absolute Error: 15.18
Accuracy: 83.36 %.


In [18]:
y_test

array([340,  46, 201, 374, 111, 355, 295, 169, 303,  84,  70,  68, 140,
       160, 222, 434, 215, 133,   7,  31, 276, 336, 263, 314,  99, 370,
       227, 273, 206, 126, 327, 250, 422, 284, 142, 192, 136, 436, 313,
       428, 168, 171, 337, 230, 204, 367, 157,  25, 292,  64, 262, 196,
         8, 254, 299,  75, 368, 361, 187,  22, 259,  93, 256,  16, 363,
       184, 249,  33,  20, 366,   3,  95, 210,  52, 182, 281, 128,  92,
       382, 349,  49, 411, 158, 224, 383, 421, 387, 351,  51, 195, 296,
        17, 274, 202,  88, 241,  21,  91, 424,  76, 137, 258,  42, 243,
       392, 309,  72, 401, 350, 233, 221], dtype=int64)

In [19]:
errors

array([40.978, 13.345, 15.298,  3.496,  5.223,  4.814, 10.087,  5.352,
       11.928,  5.092, 20.373, 21.229, 11.818, 22.435, 23.073,  6.397,
       75.421,  7.993,  5.419,  0.311, 23.737, 10.455, 70.082, 11.186,
        1.408,  4.801, 15.572,  1.416, 80.307, 22.011, 17.542,  0.298,
        2.132, 27.873,  2.791, 17.038, 17.624,  5.647, 11.63 ,  0.759,
        2.194,  3.127,  5.555, 18.62 ,  6.471, 12.928,  3.224,  1.605,
        3.881, 14.032, 40.795, 13.617, 11.993, 12.799,  6.048,  9.289,
       24.818, 90.356, 11.224, 29.709, 17.958, 33.739, 66.123, 12.811,
        4.266,  6.686,  8.034,  0.381,  7.466, 12.271,  9.493, 12.821,
        2.464,  1.036,  4.625,  0.304, 73.763,  6.485, 10.496, 11.047,
       13.929,  3.025, 41.294, 74.578, 11.585,  7.557,  9.705, 14.103,
       27.451,  4.797,  0.377,  8.632,  0.171, 10.129, 22.071, 12.931,
        6.644,  5.717,  1.348,  4.623, 22.152, 18.727, 21.149, 12.988,
       12.529, 10.72 ,  3.138,  4.61 ,  1.114, 13.688,  4.333])

In [20]:
from sklearn import metrics
# https://machinelearningmastery.com/regression-metrics-for-machine-learning

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, rf_predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, rf_predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, rf_predictions)))

Mean Absolute Error: 15.178018018018015
Mean Squared Error: 558.1043337477477
Root Mean Squared Error: 23.62423191868357


In [22]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances


array([2.58650890e-03, 2.47882322e-03, 6.12695816e-04, 8.42587361e-04,
       5.64675163e-04, 4.82470047e-04, 1.57300375e-03, 6.69228779e-04,
       5.81246912e-04, 8.09415984e-04, 6.03834200e-04, 8.21454724e-04,
       6.89781682e-04, 7.15873604e-04, 5.38605285e-04, 5.48582642e-04,
       4.83312149e-04, 9.04315977e-04, 1.86333778e-03, 1.95091260e-03,
       4.47178266e-03, 1.23352753e-03, 1.28664572e-03, 1.98795539e-03,
       1.51808816e-03, 3.99585117e-03, 2.51991660e-03, 1.13518670e-03,
       8.76451363e-04, 8.85346102e-04, 3.88885444e-04, 1.53216151e-03,
       6.58676334e-04, 6.43681152e-04, 8.83696570e-01, 1.88807623e-02,
       5.39678449e-02])

In [24]:
# Sort the features by their importance.
sorted_list=sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
sorted_df=pd.DataFrame(sorted_list, columns=['Importance', 'Feature'])
sorted_df

,Importance,Feature
0,0.883697,Social mobility rank
1,0.053968,Service rank
2,0.018881,Research rank
3,0.004472,Bachelor's to PhD rank
4,0.003996,Faculty accolades rank
5,0.002587,8-year graduation rate
6,0.002520,AmeriCorps/Peace Corps rank
7,0.002479,Graduation rate rank
8,0.001988,Faculty receiving significant awards
9,0.001951,Research expenditures rank


In [ ]:
sorted_df.to_csv('Feature_Importances_with_no_feature_reduction.csv')  

## Support Vector Regression

In [25]:
# Create a SVM model

# Instantiate a SVM model
from sklearn.svm import SVR

# Fit the data
regressor = SVR(kernel = 'poly')
svm_model = regressor.fit(X_train, y_train)
svm_model

#Testing different kernels for improved r2 score
#rbf kernel r2 score = .103
#linear kernel r2 score = -209.628
#sigmoid kernel r2 = -.013
#poly(polynomial) kernel r2= .313


SVR(kernel='poly')

In [26]:
# Predicting the Test set results
svm_predictions = svm_model.predict(X_test)

In [27]:
from sklearn.metrics import r2_score
r2_svm=r2_score(y_test, svm_predictions)
print('The r2 score for this SVM model is:', r2_svm)

The r2 score for this SVM model is: 0.3129747845868047


## Decision Tree Regression

In [29]:
# Training the Decision Tree Regression model on the Training set
from sklearn.tree import DecisionTreeRegressor
dt_model = DecisionTreeRegressor(random_state = 0)
dt_model = dt_model.fit(X_train, y_train)
dt_model

DecisionTreeRegressor(random_state=0)

In [30]:
# Predicting the Test set results
dt_predictions = dt_model.predict(X_test)

In [31]:
# Evaluate the performance.
from sklearn.metrics import r2_score
r2_dt= r2_score(y_test,dt_predictions)
print('The r2 score for this Decision Tree model is:', r2_dt)

The r2 score for this Decision Tree model is: 0.9178296794697223


## Multiple Linear Regression

In [32]:
# Training the multiple regression model on the Training set
from sklearn.linear_model import LinearRegression
mlr_model = LinearRegression()
mlr_model = mlr_model.fit(X_train, y_train)
mlr_model

LinearRegression()

In [33]:
# Predicting the Test set results
mlr_predictions = mlr_model.predict(X_test)

In [34]:
# Evaluate the performance.
from sklearn.metrics import r2_score
r2_mlr= r2_score(y_test,mlr_predictions)
print('The r2 score for this Multiple Linear Regression model is:', r2_mlr)

The r2 score for this Multiple Linear Regression model is: 0.9763915787643959
